In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt



In [41]:
with open('../sf_vs_nyc_house_data.csv', 'r') as f:
    labels = f.readline()

print(labels)

full_data = np.genfromtxt('../sf_vs_nyc_house_data.csv', delimiter=',')[1:]
full_data.shape

in_sf,beds,bath,price,year_built,sqft,price_per_sqft,elevation



(492, 8)

In [42]:
# Sample data
X = torch.tensor(full_data[::2,1:], dtype=torch.float32)  # 12 samples, each with 7 features

X= torch.nn.functional.normalize(X, dim=0)

y = torch.squeeze(torch.tensor(full_data[::2,:1], dtype=torch.float32)) # Ensure y is 1-dimensional
torch.bincount(y.to(torch.int64))

tensor([112, 134])

In [43]:
# Define the model
class MLP_Large_Adam(nn.Module):
    def __init__(self):
        super(MLP_Large_Adam, self).__init__()
        self.l0 = nn.Linear(7, 1000)  # Increase the number of neurons
        self.relu0 = nn.ReLU()
        self.l1 = nn.Linear(1000, 800)  # Increase the number of neurons
        self.relu1 = nn.ReLU()
        self.l2 = nn.Linear(800, 600)  # Increase the number of neurons
        self.relu2 = nn.ReLU()
        self.l3 = nn.Linear(600, 400)  # Increase the number of neurons
        self.relu3 = nn.ReLU()
        self.l4 = nn.Linear(400, 200)  # Increase the number of neurons
        self.relu4 = nn.ReLU()
        self.l5 = nn.Linear(200, 100)  # Increase the number of neurons
        self.relu5 = nn.ReLU()
        self.l6 = nn.Linear(100, 50)  # Increase the number of neurons
        self.relu6 = nn.ReLU()
        self.l7 = nn.Linear(50, 1)
        self.out = nn.Sigmoid()


    def forward(self, X):
        x = self.l0(X)
        x = self.relu0(x)
        x = self.l1(x)
        x = self.relu1(x)
        x = self.l2(x)
        x = self.relu2(x)
        x = self.l3(x)
        x = self.relu3(x)
        x = self.l4(x)
        x = self.relu4(x)
        x = self.l5(x)
        x = self.relu5(x)
        x = self.l6(x)
        x = self.relu6(x)
        x = self.l7(x)
        x = self.out(x)
        return x

# Initialize the model
model_l_a = MLP_Large_Adam()
criterion = nn.BCELoss()
optimizer = optim.Adam(params=model_l_a.parameters(), lr=0.001)  # Adjusted learning rate




In [44]:
# Training loop
epochs = 10000
lossi = []
for epoch in tqdm(range(epochs)):
    optimizer.zero_grad()
    output = model_l_a(X).squeeze()
    loss = criterion(output, y)
    lossi.append(loss.item())
    loss.backward()
    optimizer.step()

    # if (epoch + 1) % 100 == 0:
    #     print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation on training data to check overfitting
plt.plot(lossi)

model_l_a.eval()
with torch.no_grad():
    predictions = model_l_a(X).squeeze()
    predictions = (predictions >= 0.2).float()  # Convert probabilities to binary predictions
    accuracy = (predictions == y).float().mean()
    print(f'Accuracy on training data: {accuracy:.4f}')

cm = confusion_matrix(y.numpy(), predictions.numpy())
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])

# Plot confusion matrix
disp.plot(cmap=plt.cm.Blues)
plt.show()

  8%|▊         | 845/10000 [00:06<01:08, 133.80it/s]


KeyboardInterrupt: 

In [ ]:
sum(p.numel() for p in model_l_a.parameters())

1635201

In [ ]:
print(1635201/ 563701 )

2.9008304047713236
